In [1]:
import os
import numpy as np
import pandas as pd
import datetime

import sys
sys.path.append('../')

from utils.forecast_ingestion import get_one_forecast
from utils.measurement_ingestion import get_one_measurement

C:\Users\karim.BENDIOUB\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\karim.BENDIOUB\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.BNVRK7633HSX7YVO2TADGR4A5KEKXJAW.gfortran-win_amd64.dll
C:\Users\karim.BENDIOUB\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


### Forecast data

In [2]:
main_dir = '../data/raw/forecasts/'
start_date = pd.to_datetime('2020-05-01 00:00:00')
end_date = pd.to_datetime('2020-11-01 09:00:00')
sub_dir = os.listdir(main_dir)

# Concat files
forecast = pd.DataFrame()
for file_path in sub_dir:
    DAY = file_path[0:10]
    HOUR = file_path[11:13]
    file_path_date = pd.to_datetime(DAY.replace('_','-') + ' ' + HOUR + ':00:00')
    if (start_date <= file_path_date) & (file_path_date + datetime.timedelta(hours=8) <= end_date):
        forecast_part = get_one_forecast(main_dir,DAY,HOUR)
        forecast_part['file_creation_date'] = file_path_date + datetime.timedelta(hours=8)
        forecast = forecast.append(forecast_part)
        del forecast_part
forecast = forecast.reset_index(drop=True)

# Change to dt 
forecast['f_date']= pd.to_datetime(forecast['f_date'],format='%Y-%m-%d %H:%M:%S')
forecast['p_date']= pd.to_datetime(forecast['p_date'],format='%Y-%m-%d %H:%M:%S')

# f_period here is false
forecast = forecast[[x for x in forecast.columns if x != 'f_period' ]]
forecast.head()

,p_date,f_date,speed,temp,rad,precip,cos_wind_dir,sin_wind_dir,wind_dir,file_creation_date
0,2020-05-01,2020-05-01 00:00:00,5.7,16.4,0,0.0,0.642788,0.766044,50,2020-05-01 08:00:00
1,2020-05-01,2020-05-01 01:00:00,5.5,16.4,0,0.0,0.642788,0.766044,50,2020-05-01 08:00:00
2,2020-05-01,2020-05-01 02:00:00,5.0,16.5,0,0.0,0.601815,0.798636,53,2020-05-01 08:00:00
3,2020-05-01,2020-05-01 03:00:00,4.7,16.9,0,0.0,0.529919,0.848048,58,2020-05-01 08:00:00
4,2020-05-01,2020-05-01 04:00:00,4.4,17.1,0,0.0,0.500000,0.866025,60,2020-05-01 08:00:00


### Measurement Data

In [3]:
main_dir = '../data/raw/measurements/'
sub_dir = os.listdir(main_dir)

data_measurement = pd.DataFrame()
for file_path in sub_dir:
    if file_path[15:20] in ('06-20','07-20','08-20','09-20','10-20'):
        data_measurement_part = pd.read_csv(main_dir + file_path)
        data_measurement = data_measurement.append(data_measurement_part)
        del data_measurement_part
data_measurement.head()   

,datetime,speed,wind_dir,temp,radiation,precip,ctime,mtime
0,2020-06-01 00:00:00,3.3,230.7,20.9,0,0.0,2020-06-01 01:05:01,2020-06-02 01:05:28
1,2020-06-01 00:01:00,3.1,237.2,20.9,0,0.0,2020-06-01 01:05:01,2020-06-02 01:05:28
2,2020-06-01 00:02:00,3.0,244.8,20.9,0,0.0,2020-06-01 01:05:01,2020-06-02 01:05:28
3,2020-06-01 00:03:00,3.1,236.7,20.9,0,0.0,2020-06-01 01:05:01,2020-06-02 01:05:28
4,2020-06-01 00:04:00,3.5,254.8,20.9,0,0.0,2020-06-01 01:05:01,2020-06-02 01:05:28


### Process measurement

In [4]:
def get_season(month):
    if month in [12,1,2]:
        return 1
    if month in [3,4,5]:
        return 2
    if month in [6,7,8]:
        return 3
    if month in [9,10,11]:
        return 4
    

##########################
#### Process and save ####
##########################
measurement = data_measurement.reset_index(drop=True)

# Date format
measurement['datetime'] = pd.to_datetime(measurement['datetime'],format='%Y-%m-%d %H:%M:%S')

# Skip incomplete hours
measurement['Id_hour'] = measurement['datetime'].map(lambda x : str(x)[0:13])
measurement = measurement.merge(measurement.groupby(['Id_hour'])['datetime'].count().reset_index() \
                                .rename(columns={'datetime':'Id_hour_count'}),
                                how='left')
measurement = measurement.loc[measurement['Id_hour_count'] >= 40,].reset_index(drop=True)


# Drop na
measurement = measurement.set_index('datetime') \
              [['speed','temp', 'radiation', 'precip','wind_dir']] \
              .dropna(axis=0, how='all')

# Smooth wind direction 
measurement['cos_wind_dir'] = np.cos(2 * np.pi * measurement['wind_dir'] / 360)
measurement['sin_wind_dir'] = np.sin(2 * np.pi * measurement['wind_dir'] / 360)

# Init output measurement data
measurement_out = pd.DataFrame()
# Speed weighted hourly mean for sin & cos
measurement_out['cos_wind_dir'] = (measurement['cos_wind_dir'] * measurement['speed']).resample('H', label='right').sum() \
                                                   / measurement['speed'].resample('H', label='right').sum()
# Speed weighted hourly mean for sin & cos
measurement_out['sin_wind_dir'] = (measurement['sin_wind_dir'] * measurement['speed']).resample('H', label='right').sum() \
                                                   / measurement['speed'].resample('H', label='right').sum()

# Hourly mean for speed, temperature, radiation and precipitation
for col in ['speed','temp','radiation','precip','wind_dir']:
    measurement_out[col] = measurement[col].resample('1H', label='right').mean()

# Add caterogical features
measurement_out['season'] = measurement_out.index.month.map(get_season) # ordinal not categorical for linear models

measurement_out = measurement_out.reset_index()
# Select columns
measurement_out = measurement_out[['datetime','speed','cos_wind_dir','sin_wind_dir','wind_dir','temp','radiation','precip','season']]

# Build date Index and fill na
Idx_Measurement = pd.DataFrame(pd.date_range(measurement_out.datetime[0],
                                             measurement_out.datetime.iloc[-1],
                                             freq='H'),
                                             columns=['datetime'])

measurement_out = Idx_Measurement.merge(measurement_out,how='left')

print(measurement_out.count() / measurement_out.shape[0])
measurement_out = measurement_out.fillna(method='ffill')

datetime        1.000000
speed           0.993236
cos_wind_dir    0.986472
sin_wind_dir    0.986472
wind_dir        0.993236
temp            0.993236
radiation       0.993236
precip          0.993236
season          1.000000
dtype: float64


### Join 49 lag measurements and forecast

In [5]:
def get_f_period(p_date,f_date):
    d = pd.to_datetime(p_date) - pd.to_datetime(f_date)
    return d.days * 24  + d.seconds // 3600

# Save a copy of measurements to score results
Y_real = measurement_out.copy()
display(Y_real.loc[Y_real['datetime'] == '2020-06-02 01:00:00'])

# 49 lag of measurements horizontal stack 
df_out = Y_real.add_suffix('_t-0')
for i in range(1, 49):
    df_temp = Y_real.copy().add_suffix('_t-'+str(i))
    df_out = pd.concat([df_out,df_temp.shift(i)],axis=1)
df_out = df_out.dropna(how='any')
display(df_out.head(1))

# join measurements & forecast
df_joined = df_out.copy()
df_joined = df_joined.merge(forecast.add_suffix('_forecast'),
                 how='left',
                 left_on = 'datetime_t-0',
                 right_on='f_date_forecast')

# filter forecast files created after prediction time (same as crop out f_period > 7)
df_joined = df_joined.loc[df_joined['datetime_t-0'] >= df_joined['file_creation_date_forecast'],]


# Compute f_period
df_joined['f_period'] = df_joined[['datetime_t-0','p_date_forecast']] \
                         .apply(lambda row : get_f_period(row['datetime_t-0'],row['p_date_forecast']),axis=1)

# assert that file_creation_date_forecast is doing the job
assert((df_joined.f_period > 7).any()) 

# keep last forecast
df_joined = df_joined.groupby('datetime_t-0')['f_period'].min().reset_index() \
             .merge(df_joined,how='left')
    
# compute cos day and hour 
df_joined['cos_day'] = np.cos(2 * np.pi * df_joined['datetime_t-0'].dt.day / 365)
df_joined['cos_hour'] =  np.cos(2 * np.pi * df_joined['datetime_t-0'].dt.hour / 24)
    
display(df_joined.head(1))

,datetime,speed,cos_wind_dir,sin_wind_dir,wind_dir,temp,radiation,precip,season
48,2020-06-02 01:00:00,2.201667,-0.906926,-0.014689,184.208333,20.536667,0.0,0.0,3


,datetime_t-0,speed_t-0,cos_wind_dir_t-0,sin_wind_dir_t-0,wind_dir_t-0,temp_t-0,radiation_t-0,precip_t-0,season_t-0,datetime_t-1,...,season_t-47,datetime_t-48,speed_t-48,cos_wind_dir_t-48,sin_wind_dir_t-48,wind_dir_t-48,temp_t-48,radiation_t-48,precip_t-48,season_t-48
48,2020-06-02 01:00:00,2.201667,-0.906926,-0.014689,184.208333,20.536667,0.0,0.0,3,2020-06-02,...,2.0,2020-05-31 01:00:00,3.915,-0.738783,-0.62539,220.29,21.415,0.0,0.0,2.0


,datetime_t-0,f_period,speed_t-0,cos_wind_dir_t-0,sin_wind_dir_t-0,wind_dir_t-0,temp_t-0,radiation_t-0,precip_t-0,season_t-0,...,speed_forecast,temp_forecast,rad_forecast,precip_forecast,cos_wind_dir_forecast,sin_wind_dir_forecast,wind_dir_forecast,file_creation_date_forecast,cos_day,cos_hour
0,2020-06-02 01:00:00,13,2.201667,-0.906926,-0.014689,184.208333,20.536667,0.0,0.0,3,...,1.6,18.3,0,0.0,-0.956305,-0.292372,197,2020-06-01 20:00:00,0.999407,0.965926


### Compute forecast scenario

In [6]:
def direction_scenario(wind_dir):
    if (wind_dir <= 101.25) or (303.75 < wind_dir):
        return 'f'
    if (wind_dir <= 213.75) and (146.25 < wind_dir):
        return 'u1'
    if    ((wind_dir <= 146.25) and (101.25 < wind_dir)) \
       or ((wind_dir <= 303.75) and (213.75 < wind_dir)):
        return 'u2'

def speed_scenario(speed):
    if speed < 0.5:
        return 0
    if (speed < 1 and 0.5 <= speed):
        return 1
    if (speed < 2 and 1 <= speed):
        return 2
    if (speed < 4 and 2 <= speed):
        return 3
    if (4 <= speed):
        return 4
    
scenario_data = pd.DataFrame([['S3','S4','S4'],
                              ['S2','S3b','S2b'],
                              ['S1' ,'S3b','S2b'],
                              ['S1' ,'S3b','S2' ],
                              ['S1' ,'S1','S1']],
                            columns={'f','u1','u2'})

scenario_data = pd.DataFrame([[4,6,6],
                              [2,5,4],
                              [1 ,5,4],
                              [1 ,5,3 ],
                              [1 ,1,1]],
                            columns={'f','u1','u2'})

df_joined['scenario_forecast'] = df_joined.apply(lambda row : scenario_data.loc[speed_scenario(row['speed_forecast']), 
                                                                       direction_scenario(row['wind_dir_forecast'])
                                                                      ],
                                        axis=1)

df_joined['dangerous_forecast'] = df_joined['scenario_forecast'] > 3 

df_joined = df_joined.rename(columns={'f_period': 'f_period_forecast'})

Y_real['scenario_real'] = Y_real.apply(lambda row : scenario_data.loc[speed_scenario(row['speed']), 
                                                                     direction_scenario(row['wind_dir'])
                                                              ],
                                axis=1)
Y_real['dangerous_real'] = Y_real['scenario_real'] > 3

### Load xgboost classification 

In [7]:
pred_max = 4
xgb_classification = dict()
for i in range(1,pred_max + 1):
    for param in ['scenario','dangerous']:
        xgb_classification[param + '_' + str(i)] = pickle.load(open('../Xgboost/' + param + '_t_' + str(i), 'rb'))
#x_test = df_joined.set_index('datetime_t-0')[xgb_class.get_booster().feature_names].copy()

NameError: name 'pickle' is not defined

In [ ]:
Y_test = df_joined[['datetime_t-0']].copy()
for pred_period in range(1,pred_max + 1):
    for param in ['scenario','dangerous']:
        Y_test[param + '_predicted_' + str(pred_period)] = xgb_classification[param + '_' + str(pred_period)].predict(x_test)
        
        # join with real measurements param
        Y_test = Y_test.merge(Y_real.set_index('datetime').shift(-pred_period).reset_index()[['datetime',param + '_real']] \
                              .rename(columns={'datetime' : 'datetime_t-0', 
                                               param + '_real' : param + '_real_' + str(pred_period)}),
                             how='left')
        #display(Y_test.head(2))
        
Y_test = Y_test.dropna().reset_index(drop=True)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test['scenario_real_1'], Y_test['scenario_predicted_1'])

In [ ]:
confusion_matrix(Y_test['dangerous_real_1'].map(int), Y_test['dangerous_predicted_1'].map(int))

In [ ]:
Y_test['dangerous_real_1'].map(int).sum()

In [13]:
Y_test = df_joined[['datetime_t-0']].copy()
xgb_models[param + '_' + str(pred_period)].predict(x_test)

In [18]:
Y_test['scenario_predicted_' + str(pred_period)] = xgb_class.predict(x_test)

In [20]:
Y_test.scenario_predicted_1.value_counts()

1.0    3183
2.0     290
4.0      88
3.0      84
6.0       2
5.0       1
Name: scenario_predicted_1, dtype: int64

In [ ]:
pred_max = 3

### Load xgboost models (regression for now)

In [ ]:
import pickle
xgb_models = dict()
for i in range(1,pred_max + 1):
    for feature in ['speed','cos_wind_dir','sin_wind_dir']:
        xgb_models[feature + '_' + str(i)] = pickle.load(open('../trained_models_26072020/' + feature + '_t_' + str(i), 'rb'))

### Predictions

In [ ]:
params = ['cos_wind_dir','sin_wind_dir','speed']
pred_periods = list(range(1,pred_max + 1))


#display(Y_real.head(2))
#display(Y_real.set_index('datetime').shift(-1).reset_index().head(2))

Y_test = df_joined[['datetime_t-0']].copy()

    
for pred_period in pred_periods:
    for param in params:
        # predict 
        x_test = df_joined.set_index('datetime_t-0')[xgb_models[param + '_' + str(pred_period)].get_booster().feature_names].copy()
        Y_test[param + '_prediction_' + str(pred_period)] = xgb_models[param + '_' + str(pred_period)].predict(x_test)
        
        # join with real measurements param
        Y_test = Y_test.merge(Y_real.set_index('datetime').shift(-pred_period).reset_index()[['datetime',param]] \
                              .rename(columns={'datetime' : 'datetime_t-0', 
                                               param : param + '_real_' + str(pred_period)}),
                             how='left')
    # add real wind_dir
    Y_test = Y_test.merge(Y_real.set_index('datetime').shift(-pred_period).reset_index()[['datetime','wind_dir']] \
                  .rename(columns={'datetime' : 'datetime_t-0', 
                                   'wind_dir' : 'wind_dir' + '_real_' + str(pred_period)}),
                 how='left')
    
    # add official forecast speed & wind_dir
    Y_test = Y_test.merge(df_joined.set_index('datetime_t-0')[['speed_forecast','wind_dir_forecast']].shift(-pred_period) \
                          .rename(columns = {'speed_forecast'    : 'speed_forecast_'    + str(pred_period) , 
                                             'wind_dir_forecast' : 'wind_dir_forecast_' + str(pred_period)}) \
                          .reset_index())
        
# drop 6 last rows (we don't have real measurements yet)
Y_test = Y_test.dropna()
Y_test.shape

### Load xgboost models - classification

### Speed performance

In [ ]:
from sklearn.metrics import mean_absolute_error

for pred_period in pred_periods:
    print(pred_period, 
          mean_absolute_error(Y_test['speed_prediction_' + str(pred_period)],
                              Y_test['speed_real_'       + str(pred_period)]),
          mean_absolute_error(Y_test['speed_forecast_' + str(pred_period)],
                              Y_test['speed_real_'       + str(pred_period)]))

### Angle performance

In [ ]:
def get_angle_in_degree(cos, sin):
    #check if cos within reasonable range: 
    if (cos>=-1) & (cos <=1): 
        angle = 360 * np.arccos(cos) / (2*np.pi)
        if sin < 0:
            angle = 360 - angle
    #check if sin within reasonable range:       
    elif (sin>=-1) & (sin <=1):
        angle = 360 * np.arcsin(sin) / (2*np.pi)
        if cos < 0:
            angle = 180 - angle
        if angle < 0:
            angle += 360
    else:
        angle=0 
        #print('cos and sin out of range, returned 0')
    #because we care about the reverse angle for the scenarios
    return angle #(angle + 180) % 360

def angle_diff(x):
    if x <= 180:
        return x 
    else:
        return 360-180

In [ ]:
for i in range(1,pred_max + 1):
    x,y = 'cos_wind_dir_prediction_' + str(i),'sin_wind_dir_prediction_' + str(i)
    # compute predicted wind_dir
    Y_test['wind_dir_prediction_' + str(i)] = Y_test[[x,y]].apply(lambda row : get_angle_in_degree(row[x],row[y]),axis=1)
    print(i,
          ((Y_test['wind_dir_prediction_' + str(i)] - Y_test['wind_dir_real_' + str(i)]).abs().map(angle_diff)).mean(),
          ((Y_test['wind_dir_forecast_' + str(i)] - Y_test['wind_dir_real_' + str(i)]).abs().map(angle_diff)).mean())

###  Compute scenario

In [ ]:
def direction_scenario(wind_dir):
    if (wind_dir <= 101.25) or (303.75 < wind_dir):
        return 'f'
    if (wind_dir <= 213.75) and (146.25 < wind_dir):
        return 'u1'
    if    ((wind_dir <= 146.25) and (101.25 < wind_dir)) \
       or ((wind_dir <= 303.75) and (213.75 < wind_dir)):
        return 'u2'

def speed_scenario(speed):
    if speed < 0.5:
        return 0
    if (speed < 1 and 0.5 <= speed):
        return 1
    if (speed < 2 and 1 <= speed):
        return 2
    if (speed < 4 and 2 <= speed):
        return 3
    if (4 <= speed):
        return 4
    
scenario_data = pd.DataFrame([['S3','S4','S4'],
                              ['S2','S3b','S2b'],
                              ['S1' ,'S3b','S2b'],
                              ['S1' ,'S3b','S2' ],
                              ['S1' ,'S1','S1']],
                            columns={'f','u1','u2'})

for i in range(1,pred_max + 1):
    for cat in ('prediction','forecast','real'):
        speed , wind_dir  = 'speed_' + cat + '_' + str(i), 'wind_dir_' + cat + '_' + str(i)
        scenario = 'scenario_' + cat + '_' + str(i)
        #print(speed,wind_dir,scenario)
        Y_test[scenario] = Y_test.apply(lambda row : scenario_data.loc[speed_scenario(row[speed]), 
                                                                       direction_scenario(row[wind_dir])
                                                                      ],
                                        axis=1)

In [ ]:
i = 1
x,y = 'scenario_' + 'prediction' + '_' + str(i), 'scenario_' + 'real' + '_' + str(i)

In [ ]:
from sklearn.metrics import confusion_matrix
scenario = ['S1','S2','S2b','S3','S3b','S4']
pd.DataFrame(confusion_matrix(Y_test[y],Y_test[x], labels=scenario),
             columns = scenario,
             index   = scenario)

In [ ]:
Y_test[y].value_counts()